In [177]:
def get_spark_env(job_name, g, n):
    from pyspark.sql import SparkSession
    return SparkSession.builder.master("local[*]").config('spark.executor.memoryOverhead.config', n).config('spark.driver.memory', g).appName(job_name).getOrCreate()

def Print_DF_Rows(DF,NbRows):
    import pandas as pd
    return pd.DataFrame(DF.take(NbRows),columns=DF.columns)

job_name = 'Formation'
spark = get_spark_env(job_name, '4g', '3')
print (spark.version)

df = spark.read.csv("Dataset_9_cost.csv", inferSchema = 'true', header=True, sep="|")
df

3.1.1


DataFrame[_c0: int, Unnamed: 0: int, Unnamed: 0.1: int, Unnamed: 0.1.1: int, Unnamed: 0.1.1.1: int, Unnamed: 0.1.1.1.1: string, Unnamed: 0.1.1.1.1.1: string, Unnamed: 0.1.1.1.1.1.1: string, Unnamed: 0.1.1.1.1.1.1.1: string, age: string, sex: string, bmi: string, children: int, smoker: string, region: string, charges: double]

# clean collumns

In [178]:
df.columns
def keepData(dataframe, columns):
    notIn = list(set(dataframe.columns)-set(columns))
    return dataframe.drop(*notIn)
df = keepData(df, ["age", "sex", "bmi", "children", "smoker", "region", "charges"])
df.printSchema()

root
 |-- age: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: string (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



In [179]:
df.show(20,False)

+--------------------+--------------------+------+--------+-----------+---------+------------------+
|age                 |sex                 |bmi   |children|smoker     |region   |charges           |
+--------------------+--------------------+------+--------+-----------+---------+------------------+
|Allez au boulot ! :)|female              |None  |0       |yes        |southwest|16884.924         |
|18                  |male                |33.77 |1       |no         |southeast|1725.5523         |
|28                  |male                |33.0  |3       |no         |southeast|4449.462          |
|33                  |male                |22.705|0       |no         |northwest|21984.47061       |
|32                  |male                |28.88 |0       |no         |northwest|3866.8552         |
|31                  |female              |25.74 |0       |None       |null     |3756.6216         |
|46                  |female              |33.44 |1       |no         |southeast|8240.5896 

# recherches des valeurs bizarre

In [180]:
def weirdDecialValue(dataframe, column, ranges):
    from pyspark.sql import functions as F
    
    weird = dataframe.filter((F.col(column).cast("double") < ranges["min"]) | (F.col(column).cast("double").isNull()) | (F.col(column).cast("double") > ranges["max"]))
    return weird

def notInListValue(dataframe, column, acceptedValue):
    return dataframe.filter(~dataframe[column].isin(acceptedValue))

import math
# mapping des columns avec les valeurs min est max
numberMapping = {"age": {"min": 0, "max": 120}, "children": {"min": 0, "max": 20}, "bmi": {"min": 0, "max": math.inf}, "charges": {"min": 0, "max": math.inf}}

for key in numberMapping:
    display(numberMapping[key])
    weird = weirdDecialValue(df, key, numberMapping[key])
    display(f"il y a {weird.count()} données qui on des valeurs décimales bizarres")
    weird.show()

# trouver toutes les données uniques
for column in ["sex", "smoker", "region"]:
    display(f'{column} valeurs uniques {df.select(column).distinct().collect()}')


stringMapping = {"sex": ["male","female"], "smoker": ["yes","no"],"region": ["northwest","southeast","southwest","northeast"]}
for key in stringMapping:
    display(key)
    notInListValue(df,key, stringMapping[key]).show()


{'min': 0, 'max': 120}

'il y a 139 données qui on des valeurs décimales bizarres'

+--------------------+--------------------+--------------------+--------+------+---------+------------------+
|                 age|                 sex|                 bmi|children|smoker|   region|           charges|
+--------------------+--------------------+--------------------+--------+------+---------+------------------+
|Allez au boulot ! :)|              female|                None|       0|   yes|southwest|         16884.924|
|Allez au boulot ! :)|              female|               30.78|       1|    no|northeast|        10797.3362|
|Allez au boulot ! :)|                male|                40.3|       0|    no|     null|         10602.385|
|         ù*ùfsfsf///|              female|              32.965|       3|    no|northwest|       15612.19335|
|         ù*ùfsfsf///|                null|               36.67|       1|   yes|northeast|        39774.2763|
|         55555555555|              female|                30.8|       2|    no|southeast|          6313.759|
|Allez au 

{'min': 0, 'max': 20}

'il y a 15 données qui on des valeurs décimales bizarres'

...

+--------------------+--------------------+--------------------+--------+------+---------+------------------+
|                 age|                 sex|                 bmi|children|smoker|   region|           charges|
+--------------------+--------------------+--------------------+--------+------+---------+------------------+
|                  60|Allez au boulot ! :)|               25.84|       0|    no|northwest|28923.136919999997|
|                  56|          courage!!!|               39.82|       0|    no|southeast|        11090.7178|
|                  30|Allez au boulot ! :)|                35.3|       0|   yes|southwest|36837.467000000004|
|                  18|               hello|                34.1|       0|  None|southeast|          1137.011|
|                  59|Allez au boulot ! :)|               27.72|       3|    no|southeast|        14001.1338|
|                  26|               hello|                20.8|       0|  None|southwest|            2302.3|
|         

'smoker'

+----------+----------+--------------------+--------+-----------+---------+------------------+
|       age|       sex|                 bmi|children|     smoker|   region|           charges|
+----------+----------+--------------------+--------+-----------+---------+------------------+
|        31|    female|               25.74|       0|       None|     null|         3756.6216|
|        37|      male|               29.83|       2|ù*ùfsfsf///|northeast|         6406.4107|
|        18|     hello|                34.1|       0|       None|southeast|          1137.011|
|        26|     hello|                20.8|       0|       None|southwest|            2302.3|
|        60|    female|               24.53|       0|       None|southeast|        12629.8967|
|         0|    female|                None|       1|       None|southeast|        23244.7902|
|        40|    female|               36.19|       0|       None|southeast|              null|
|        31|      male|                28.5|      

'region'

+--------------------+------+-----------+--------+------+------+------------------+
|                 age|   sex|        bmi|children|smoker|region|           charges|
+--------------------+------+-----------+--------+------+------+------------------+
|                  37|female|      27.74|       3|    no|  None|         7281.5056|
|                  40|female|      28.69|       3|    no| hello|         8059.6791|
|Allez au boulot ! :)|  male|ù*ùfsfsf///|       2|   yes|  None|        34303.1672|
|                  22|female|     39.805|       0|    no| hello|        2755.02095|
|                  22|  male|      37.62|       1|   yes| hello|        37165.1638|
|                  18| hello|     30.115|       0|    no| hello|        21344.8467|
|                  20|  male|     28.025|       1|   yes| hello|       17560.37975|
|                  37|  male|       30.8|       0|  None| hello|          4646.759|
|                  21|female|      35.72|       0|    no|  None|         240

# clean data

In [181]:
def replaceBadValue(dataframe, column, badValue, replace):
    from pyspark.sql.functions import when
    
    return dataframe.withColumn(column, when(dataframe[column].isin(badValue) | dataframe[column].isNull(), replace).otherwise(dataframe[column]))

def replaceWithMean(dataframe, column, ranges):
    from pyspark.sql.functions import mean, col, when, lit
    
    df = dataframe.withColumn(column, col(column).cast("double"))
    df = df.withColumn(column, when((col(column) < ranges["min"]) | (col(column) > ranges["max"]), None).otherwise(col(column)))
    mean = df.agg(mean(col(column))).collect()[0][0]
    df = df.withColumn(column, when(col(column).isNull(), mean).otherwise(col(column)))
    return df

# pour les chiffre numerique
modify=df
for key in numberMapping:
    modify = replaceWithMean(modify, key, numberMapping[key])
    
# pour les données
for key in stringMapping:
    notInList = notInListValue(modify, key, stringMapping[key])
    badValue = [row[0] for row in notInList.select(key).distinct().collect()]
    modify = replaceBadValue(modify, key, badValue, None)

modify.show()
modify.printSchema()


+-----------------+------+-----------------+--------+------+---------+------------------+
|              age|   sex|              bmi|children|smoker|   region|           charges|
+-----------------+------+-----------------+--------+------+---------+------------------+
|38.96413678065054|female|30.65886130136987|     0.0|   yes|southwest|         16884.924|
|             18.0|  male|            33.77|     1.0|    no|southeast|         1725.5523|
|             28.0|  male|             33.0|     3.0|    no|southeast|          4449.462|
|             33.0|  male|           22.705|     0.0|    no|northwest|       21984.47061|
|             32.0|  male|            28.88|     0.0|    no|northwest|         3866.8552|
|             31.0|female|            25.74|     0.0|  null|     null|         3756.6216|
|             46.0|female|            33.44|     1.0|    no|southeast|         8240.5896|
|             37.0|female|            27.74|     3.0|    no|     null|         7281.5056|
|         

# transformation des données

In [182]:
def elementToInt(dataframe, column):
    i=1
    replace = {}
    elements = [row[0] for row in dataframe.select(column).distinct().collect()]
    for elem in elements:
        if type(elem) == float and math.isnan(elem):
            continue
        replace[elem] = i
        i+=1
    return replace

def replaceWithMap(dataframe, column, mapping):
    from pyspark.sql.functions import col, lit, create_map, when
    from itertools import chain
    
    # none cant be map
    noneMapping = mapping[None]
    del mapping[None]
    
    mapping_expr = create_map([lit(x) for x in chain(*mapping.items())])
    dataframe = dataframe.withColumn(column, mapping_expr[col(column)])
    # none cant be map
    dataframe = dataframe.withColumn(column, when(col(column).isNull(), noneMapping).otherwise(col(column)))
    return dataframe

columns = ["sex", "region", "smoker"]
modify2 = modify
mapping = {}
for column in columns:
    mapping[column] = elementToInt(modify2, column)
    modify2 = replaceWithMap(modify2, column, mapping[column])
display(mapping)
modify2.show()
modify2.printSchema()

{'sex': {'female': 2, 'male': 3},
 'region': {'northwest': 1, 'southeast': 2, 'northeast': 3, 'southwest': 5},
 'smoker': {'no': 2, 'yes': 3}}

+-----------------+---+-----------------+--------+------+------+------------------+
|              age|sex|              bmi|children|smoker|region|           charges|
+-----------------+---+-----------------+--------+------+------+------------------+
|38.96413678065054|  2|30.65886130136987|     0.0|     3|     5|         16884.924|
|             18.0|  3|            33.77|     1.0|     2|     2|         1725.5523|
|             28.0|  3|             33.0|     3.0|     2|     2|          4449.462|
|             33.0|  3|           22.705|     0.0|     2|     1|       21984.47061|
|             32.0|  3|            28.88|     0.0|     2|     1|         3866.8552|
|             31.0|  2|            25.74|     0.0|     1|     4|         3756.6216|
|             46.0|  2|            33.44|     1.0|     2|     2|         8240.5896|
|             37.0|  2|            27.74|     3.0|     2|     4|         7281.5056|
|             37.0|  3|            29.83|     2.0|     1|     3|         640